# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [14]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [15]:
# Drop unnecessary columns
new_df = df.drop(['timestamp'], axis =1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [16]:
from surprise import Reader, Dataset

In [17]:
# read in values as Surprise dataset 
reader = Reader(rating_scale=(1,5))

surprise_df = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [18]:
dataset = surprise_df.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [19]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [11]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}

g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(surprise_df)

In [12]:
# print out optimal parameters for SVD after GridSearch
g_s_svd.best_score

{'rmse': 0.8700917882497332, 'mae': 0.6688810699014648}

In [13]:
g_s_svd.best_params

{'rmse': {'n_factors': 50, 'reg_all': 0.05},
 'mae': {'n_factors': 100, 'reg_all': 0.05}}

In [22]:
# cross validating with KNNBasic
knn = KNNBasic(sim_options={'name':'Pearson',
                            'user_based':'True'})
cv_knn = cross_validate(knn, surprise_df, n_jobs = -1)

In [23]:
# print out the average RMSE score for the test set

print(np.mean(cv_knn['test_rmse']))

0.9740665215621034


In [25]:
# cross validating with KNNBaseline
knnbaseline = KNNBaseline(sim_options={'name':'Pearson','user_based':'True'})
cv_knn_baseline = cross_validate(knnbaseline, surprise_df, n_jobs = -1)

In [26]:
# print out the average score for the test set
print(np.mean(cv_knn_baseline['test_rmse']))

0.8752156843293866


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [27]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [28]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [29]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [30]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0448026374557178, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [36]:
def movie_rater(movie_df,num, genre=None):
    userId = 1001
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = move_df.sample(1)
        print(movie)
        rating = input('If you have seen this movie, rate it from 1-5; if not, enter \'n\'')
        if rating == 'n':
            continue
        else:
            rating_list.append({'userId':userId, 'movieId':movie['movieId'].values[0], 'rating':rating})
            num -= 1
    return rating_list
        

In [39]:
# try out the new function here!
user_rating = movie_rater(df_movies, 5, 'Comedy')

     movieId              title        genres
329      371  Paper, The (1994)  Comedy|Drama
If you have seen this movie, rate it from 1-5; if not, enter 'n'n
      movieId               title         genres
3029     4053  Double Take (2001)  Action|Comedy
If you have seen this movie, rate it from 1-5; if not, enter 'n'3
      movieId          title                genres
1773     2371  Fletch (1985)  Comedy|Crime|Mystery
If you have seen this movie, rate it from 1-5; if not, enter 'n'5
     movieId                                   title                    genres
863     1136  Monty Python and the Holy Grail (1975)  Adventure|Comedy|Fantasy
If you have seen this movie, rate it from 1-5; if not, enter 'n'5
      movieId                     title  \
2056     2735  Golden Child, The (1986)   

                                       genres  
2056  Action|Adventure|Comedy|Fantasy|Mystery  
If you have seen this movie, rate it from 1-5; if not, enter 'n'5
      movieId                        

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [38]:
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [41]:
## add the new ratings to the original ratings DataFrame
df2 = new_df.append(user_rating, ignore_index=True)

surprise_df2 = Dataset.load_from_df(df2[['userId', 'movieId', 'rating']], reader)

In [45]:
# train a model using the new combined DataFrame
svd_srp = SVD(n_factors= 50, reg_all= 0.05)

svd_srp.fit(surprise_df2.build_full_trainset())

In [48]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
unranked_movies = []
for i in new_df['movieId'].unique():
    unranked_movies.append((i, svd_srp.predict(1000,i)[3]))

In [49]:
unranked_movies[:5]

[(1, 3.9100633750439258),
 (3, 3.260081187800652),
 (6, 3.9548623547256843),
 (47, 3.9941296653636162),
 (50, 4.226427592182422)]

In [54]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(unranked_movies,key = lambda x:x[1], reverse = True)
ranked_movies[:5]

[(318, 4.378270380750458),
 (1204, 4.358574137264557),
 (750, 4.345506720436573),
 (904, 4.282956502948916),
 (858, 4.282141456761516)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [60]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for i in range(n):
        title = movie_title_df.loc[movie_title_df['movieId'] == user_ratings[i][0]]['title']
        print('Recommendation #',i+1,':',user_ratings[1], title)
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation # 1 : (1204, 4.358574137264557) 277    Shawshank Redemption, The (1994)
Name: title, dtype: object
Recommendation # 2 : (1204, 4.358574137264557) 906    Lawrence of Arabia (1962)
Name: title, dtype: object
Recommendation # 3 : (1204, 4.358574137264557) 602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object
Recommendation # 4 : (1204, 4.358574137264557) 686    Rear Window (1954)
Name: title, dtype: object
Recommendation # 5 : (1204, 4.358574137264557) 659    Godfather, The (1972)
Name: title, dtype: object


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.